In [1]:
import ee 
ee.Initialize()

In [2]:
import geemap
Map = geemap.Map()
# Map

In [3]:

from typing import Dict
import ee
ee.Initialize()

from easydict import EasyDict as edict
from prettyprinter import pprint

import logging
logger = logging.getLogger(__name__)

# from eo_class.modisPoly import MODIS_POLY
import yaml
import numpy as np
import json

""" CFG """
cfg = edict({
    "JSON": "G:\PyProjects\wildfire-s1s2-dataset\wildfire_events\MTBS_AK_2017_2019_events_ROI.json"
})

""" Fire Event """

from eo_class.fireEvent import load_json
print(cfg.JSON)
EVENT_SET = load_json(cfg.JSON)
event = EVENT_SET['ak6569814836520190622']
# pprint(event)

queryEvent = edict(event.copy())
queryEvent['start_date'] = event['Ig_Date']
queryEvent['end_date'] = event['modisEndDate']
queryEvent['year'] = event['YEAR']

pprint(queryEvent)

queryEvent['roi'] = ee.Geometry.Polygon(event['roi']).bounds()


# pprint(queryEvent)

period_start = ee.Date(queryEvent['start_date']).advance(-1, 'month')
period_end = ee.Date(queryEvent['start_date']).advance(2, 'month')
print(period_start.format().getInfo(), period_end.format().getInfo())



G:\PyProjects\wildfire-s1s2-dataset\wildfire_events\MTBS_AK_2017_2019_events_ROI.json
easydict.EasyDict({
    'Asmnt_Type': 'Extended',
    'BurnBndAc': 181458,
    'BurnBndLat': '65.62',
    'BurnBndLon': '-148.137',
    'Comment':
        'Area is a mixture of forest and shrubland.  High severity is mainly '
        'located in forested areas.',
    'Event_ID': 'AK6569814836520190622',
    'High_T': 700,
    'Ig_Date': '2019-06-22',
    'IncGreen_T': -150,
    'Incid_Name': 'HESS CREEK',
    'Incid_Type': 'Wildfire',
    'Low_T': 40,
    'Map_ID': 10016692,
    'Map_Prog': 'MTBS',
    'Mod_T': 363,
    'NoData_T': -970,
    'Perim_ID': None,
    'Post_ID': '807101420200705',
    'Pre_ID': '807101420170713',
    'dNBR_offst': 9,
    'dNBR_stdDv': 18,
    'irwinID': '4AB4BF76-9D3B-4C1C-90B0-4BC9EE81B40A',
    'crs': 'EPSG:3338',
    'name': 'ak6569814836520190622',
    'YEAR': 2019,
    'mtbs_pre': 'ak6569814836520190622_20170713_L8_refl.tif',
    'mtbs_post': 'ak6569814836520190622_20

In [4]:
def get_pntsFilter(roi, buffer_size=-2e3):
    ## ==> Point Filters <==
    pnt_roi = roi.buffer(buffer_size, ee.ErrorMargin(1)).bounds()
    coordList = ee.List(pnt_roi.coordinates().get(0))
    btmLeft = ee.Geometry.Point(coordList.get(0))
    btmRight = ee.Geometry.Point(coordList.get(1))
    topRight = ee.Geometry.Point(coordList.get(2))
    topLeft = ee.Geometry.Point(coordList.get(3))
    pntsFC = ee.FeatureCollection([btmLeft, btmRight, topRight, topLeft]).style(color="red")

    # if len(pntsList) > 0:
    pntsFilter = ee.Filter.And(
        ee.Filter.geometry(btmLeft),
        ee.Filter.geometry(btmRight),
        ee.Filter.geometry(topRight),
        ee.Filter.geometry(topLeft))
    
    return pntsFilter, pntsFC

pntsFilter, pntsFC = get_pntsFilter(queryEvent.roi)

In [5]:
# """ Query S2 MSI Data """

# S2_Dict = {
#     'SR': "COPERNICUS/S2_SR",
#     'TOA': "COPERNICUS/S2"
# }

# L8_Dict = {
#     'SR': "LANDSAT/LC08/C01/T1_SR",
#     'TOA': "LANDSAT/LC08/C01/T1_TOA"
# }


# MSI = ee.ImageCollection(S2_Dict['TOA']).filterBounds(queryEvent['roi'])
# cloudFilter = ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 30)

# MSI_pre = MSI.filterDate(period_start.advance(-1, 'year'), period_end.advance(-1, "year")).filter(cloudFilter).median()
# MSI_post = MSI.filterDate(period_start.advance(1, 'year'), period_end.advance(1, "year")).filter(cloudFilter).median()

In [6]:
""" Query S1 SAR Data """
def set_group_index_4_S1(img, groupLevel=13, labelShowLevel=13):
        orbitKey = (ee.String(img.get("orbitProperties_pass")).replace('DESCENDING', 'DSC').replace('ASCENDING', 'ASC')
                    .cat(ee.Number(img.get("relativeOrbitNumber_start")).int().format()))
        Date = (img.date().format().slice(0, labelShowLevel)
                    .replace('-', '').replace('-', '').replace(':', '').replace(':', ''))
        Name = (Date).cat('_').cat(orbitKey)

        groupIndex = img.date().format().slice(0, groupLevel)  # 2017 - 07 - 23T14:11:22(len: 19)
        return img.setMulti({
            'GROUP_INDEX': groupIndex,
            'IMG_LABEL': Name,
            'Orbit_Key': orbitKey
        })

def unionGeomFun(img, first):
    rightGeo = ee.Geometry(img.geometry())
    return ee.Geometry(first).union(rightGeo)

S1_flt = ee.ImageCollection(ee.ImageCollection("COPERNICUS/S1_GRD")
                        .filterBounds(queryEvent.roi)
                        .filterMetadata('instrumentMode', "equals", 'IW')
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                    ).select(['VH', 'VV'])

S1_pre = S1_flt.filterDate(period_start.advance(-1, 'year'), period_end.advance(-1, "year")).map(set_group_index_4_S1)
S1_post = S1_flt.filterDate(period_start.advance(1, 'year'), period_end.advance(1, "year")).map(set_group_index_4_S1)

# SAR orbits
pre_orbits = S1_pre.aggregate_array("Orbit_Key").distinct().getInfo()
post_orbits = S1_post.aggregate_array("Orbit_Key").distinct().getInfo()
common_orbits = list(set(pre_orbits) & set(post_orbits))

# Compute average image for each orbit
S1 = {}
for orbit in common_orbits:
# for orbit in ['DSC160']:
    orbImgCol_pre = S1_pre.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)
    orbImgCol_post = S1_post.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)

    orb_images = orbImgCol_pre.filter(ee.Filter.eq("IMG_LABEL", orbImgCol_pre.first().get("IMG_LABEL")))
    firstImgGeom = orb_images.first().geometry()
    orb_geom = ee.Geometry(orb_images.iterate(unionGeomFun, firstImgGeom))

    if orb_geom.contains(queryEvent.roi, ee.ErrorMargin(1)).getInfo():
        S1[f"{orbit}_pre"] = orbImgCol_pre.mean()
        S1[f"{orbit}_post"] = orbImgCol_post.mean()

        # Map.addLayer(orb_geom, {}, f"{orbit}_geom")
        # Map.addLayer(S1[f"{orbit}_pre"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_pre")
        # Map.addLayer(S1[f"{orbit}_post"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_post")
        

In [8]:
Map.centerObject(queryEvent.roi, 6)

def unionGeomFun(img, first):
    rightGeo = ee.Geometry(img.geometry())
    return ee.Geometry(first).union(rightGeo)

S1 = {}
for orbit in common_orbits:
# for orbit in ['DSC160']:
    orbImgCol_pre = S1_pre.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)
    orbImgCol_post = S1_post.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)

    orb_images = orbImgCol_pre.filter(ee.Filter.eq("IMG_LABEL", orbImgCol_pre.first().get("IMG_LABEL")))
    firstImgGeom = orb_images.first().geometry()
    orb_geom = ee.Geometry(orb_images.iterate(unionGeomFun, firstImgGeom))

    if orb_geom.contains(queryEvent.roi, ee.ErrorMargin(1)).getInfo():
        S1[f"{orbit}_pre"] = orbImgCol_pre.mean()
        S1[f"{orbit}_post"] = orbImgCol_post.mean()

        Map.addLayer(orb_geom, {}, f"{orbit}_geom")
        Map.addLayer(S1[f"{orbit}_pre"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_pre")
        Map.addLayer(S1[f"{orbit}_post"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_post")
        
Map.addLayer(queryEvent['roi'], {}, 'roi', True, 0.3)

In [9]:
S1 

{'DSC131_pre': <ee.image.Image at 0x1cf77c857c8>,
 'DSC131_post': <ee.image.Image at 0x1cf78fed388>,
 'ASC94_pre': <ee.image.Image at 0x1cf77c930c8>,
 'ASC94_post': <ee.image.Image at 0x1cf79003bc8>}

In [15]:
checkOrb = "ASC94"
S1_MT = ee.Image.cat([S1[f'{checkOrb}_pre'], S1[f'{checkOrb}_post']])
Map.addLayer(S1_MT, {'bands':['VH', 'VH_1', 'VH_1'], 'min': -20, 'max': -10}, f'S1_MT: {checkOrb}')

In [10]:
Map

Map(center=[65.61219222118567, -148.14545638310784], controls=(WidgetControl(options=['position', 'transparent…

In [17]:
orbImgCol_post = S1_post.filter(ee.Filter.eq("Orbit_Key", "ASC94"))

In [18]:
orbImgCol_post.aggregate_array("IMG_LABEL").getInfo()

['20200531T03_ASC94',
 '20200531T03_ASC94',
 '20200612T03_ASC94',
 '20200612T03_ASC94',
 '20200624T03_ASC94',
 '20200624T03_ASC94',
 '20200706T03_ASC94',
 '20200706T03_ASC94',
 '20200718T03_ASC94',
 '20200718T03_ASC94',
 '20200730T03_ASC94',
 '20200730T03_ASC94',
 '20200811T03_ASC94',
 '20200811T03_ASC94']

In [10]:
Map

Map(bottom=4418.0, center=[66.31860698381531, -148.87767470525787], controls=(WidgetControl(options=['position…

In [9]:
orbit

'DSC160'

In [26]:
ee.ImageCollection(s1_pre_tmp).filter(pntsFilter).size().getInfo()

1

In [13]:
orbImgCol_post.mosaic().geometry().getInfo()

{'type': 'Polygon',
 'coordinates': [[[-180, -90],
   [180, -90],
   [180, 90],
   [-180, 90],
   [-180, -90]]]}

In [4]:
Map.centerObject(queryEvent.roi, 10)
Map.addLayer(MSI_pre, {"bands": ['B12', 'B8', 'B4'], "min":0, "max": 3000}, 'MSI_pre')
Map.addLayer(MSI_post, {"bands": ['B12', 'B8', 'B4'], "min":0, "max": 3000}, 'MSI_post')

Map.addLayer(queryEvent['roi'], {}, 'roi')

In [30]:
Map.addLayer(pntsFC, {}, 'pntsFC')

In [32]:
Map.addLayer(S1_pre.filter(ee.Filter.eq("Orbit_Key", orbit)).first().geometry(), {}, 'geom')